# Retail Transaction Pattern

#### Mengheng Xue

#### Due: 2019-04-03

### Import Libraries

In [2]:
# Importing the libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


### Data Preprocessing
+ We group the retail transaction based invoice date since it different customers ID my have multiple transactions in different invoice date. But we only care one transaction  at one time period.  

In [7]:
file = 'Retail_Transaction_Example.xlsx'
data = pd.read_excel(file)
df_rt = data.copy()
display(df_rt.head())


,CustomerID,StockCode,Description,InvoiceDate
0,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00
1,17850.0,71053,WHITE METAL LANTERN,2010-12-01 08:26:00
2,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00
3,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00
4,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00


In [9]:
# Some of the descriptions have spaces that need to be removed
df_rt['Description'] = df_rt['Description'].str.strip()

# Add a column called 'Quantity'
df_rt['Quantity'] = 1

df_rt.head()


,CustomerID,StockCode,Description,InvoiceDate,Quantity
0,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,1
1,17850.0,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,1
2,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,1
3,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,1
4,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,1


In [10]:
# After the cleanup, we need to consolidate the items into 1 transaction per row with each product 1 hot encoded.
basket = (df_rt.pivot_table(index='CustomerID',
                            columns="Description", values="Quantity").fillna(0))
basket.head()


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0.
# This step will complete the one hot encoding of the data and remove the postage column (since that charge is not one we wish to explore)
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Now that the data is structured properly, we can generate frequent item sets that have a support of at least 7%
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [14]:
# generate the rules with their corresponding support, confidence and lift
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.097667,0.089204,0.074108,0.758782,8.506143,0.065396,3.775824
1,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.089204,0.097667,0.074108,0.830769,8.506143,0.065396,5.331968
2,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.131061,0.111391,0.070677,0.539267,4.841223,0.056078,1.928686
3,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.111391,0.131061,0.070677,0.634497,4.841223,0.056078,2.377377
4,(LUNCH BAG RED RETROSPOT),(LUNCH BAG CARS BLUE),0.121683,0.110018,0.070448,0.578947,5.262283,0.057061,2.113707


### Interpreting the results
- When we set the min_support equal to 0.07:  
- There is a high connection between buying (GREEN REGENCY TEACUP AND SAUCER) and buying (ROSES REGENCY TEACUP AND SAUCER). 
    - (GREEN REGENCY TEACUP AND SAUCER) -> (ROSES REGENCY TEACUP AND SAUCER) has confidence 0.830769, meaning that given that one buy (GREEN REGENCY TEACUP AND SAUCER), the probability of he also buying (ROSES REGENCY TEACUP AND SAUCER) is 0.830769.
    - (ROSES REGENCY TEACUP AND SAUCER) -> (GREEN REGENCY TEACUP AND SAUCER) has condifence 0.758782, meaning that given that one buy (ROSES REGENCY TEACUP AND SAUCER), the probability of he also buying (GREEN REGENCY TEACUP AND SAUCER) is 0.758782.
- There is a high connection between buying (HEART OF WICKER LARGE) and buying (HEART OF WICKER SMALL).  
    - (HEART OF WICKER LARGE) -> (HEART OF WICKER SMALL) has confidence 0.634497, meaning that given that one buy (HEART OF WICKER LARGE), the probability of he also buying (HEART OF WICKER SMALL) is 0.634497. 
    - (HEART OF WICKER SMALL) -> (HEART OF WICKER LARGE) has confidence 0.539267, meaning that given that one buy (HEART OF WICKER SMALL), the probability of he also buying (HEART OF WICKER LARGE) is 0.539267. 
- There is a high connection between buying (LUNCH BAG RED RETROSPOT) and buying (LUNCH BAG CARS BLUE). 
    - (LUNCH BAG RED RETROSPOT) -> (LUNCH BAG CARS BLUE) has confidence 0.578947, meaning that given that one buy (LUNCH BAG RED RETROSPOT), the probability of he also buying (LUNCH BAG CARS BLUE) is 0.578947. 

In [17]:
# We can filter the dataframe using standard pandas code. In this case, look for a large lift (6) and high confidence (.8)
rules[(rules['lift'] >= 6) &
      (rules['confidence'] >= 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.089204,0.097667,0.074108,0.830769,8.506143,0.065396,5.331968


+ It seems that the (GREEN REGENCY TEACUP AND SAUCER) and (ROSES REGENCY TEACUP AND SAUCER) are purchased together in a manner that is higher than the overall probability would suggest.